## Парсинг застройщиков

Сайт: https://наш.дом.рф/сервисы/единый-реестр-застройщиков

Зачем это нужно?

    Получение единой базы данных потенциальных партнёров
    Возможность для рассылки выгодных предложений

In [1]:
import requests
import pandas as pd
import numpy as np
import urllib3
from bs4 import BeautifulSoup

In [2]:
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36 OPR/65.0.3467.78"}

В изначальном окне список застройщиков, при нажатии на каждого - переход на персональную страницу застройщика с нужными данными.

***

Функция обхода по списку:

In [29]:
def get_info_page(url):
    #создаём датафрейм
    df = pd.DataFrame(columns=['Застройщик', 'ИНН', 'КПП', 'ОГРН', 'Полное наименование', "Юридический адрес", "Фактический адрес", "Руководитель", "Телефон", "Сайт", "E-mail"])
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text) #в нормальный вид
    #каждый класс - это застройщик
    classes = soup.find_all("div", attrs = {"class" : "styles__Container-qpwb36-3 cHZRte"})
    for cl in classes:
        link = cl.find('a') #ссылка на страницу застройщика с дополнительной информацией
        if link != None: #если такая есть, идем дальше
            link = link.get('href') 
            #имя застройщика
            name = cl.find("span",attrs={"class":"styles__Ellipsis-sc-1fw79ul-0 cDcbYl styles__Child-qpwb36-0 styles__Primary-qpwb36-1 daoZod"})
            if name != None:
                #избегаем проблем с кодировкой
                name = name.get_text().replace('\u200e', '')
                #вызываем функцию перехода по ссылке и извлечения информации о застройщике
                comp_info = get_info_company(link)
                #на первое место массива характеристик ставим имя
                comp_info.insert(0, name)
                print(comp_info) #печать для контроля
                print(len(comp_info))
                #добавляем в датафрейм
                df = df.append(pd.Series(comp_info, index=df.columns), ignore_index=True)
    return df

****

Функция получения данных о застройщике:

In [35]:
def get_info_company(link):
    link_info = [] #массив характеристик
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text)
    soup.encoding = 'utf8'
    #находим объекты первых характеристик - ИНН, КПП, ОРГН, Имя, Юр и Факт адрес
    chars = soup.find_all('div', attrs = {'class':'styles__BuilderCardRequisitesBlock-p65t3v-3 DpBEG'})
    columns = ['Застройщик', 'ИНН', 'КПП', 'ОГРН', 'Полное наименование', "Юридический адрес", "Фактический адрес", "Руководитель", "Телефон", "Сайт", "E-mail"]

    for char in chars: #извлекаем первые характеристики
        char_text = char.find('p', attrs = {'class' : 'styles__TypographyP-sc-1txyxb-4 QLiFf'})
        if (char_text) != None: #проверка на наличие информации в классе характеристики
            link_info.append(char_text.get_text())
        else:
            link_info.append('-')
    #находим объекты вторых характеристик - категория "Контакты" - руководитель, телефон, сайт, е-мейл
    contacts = soup.find_all('div', attrs = {'class' : 'styles__BuilderCardContactsBlock-erzrc1-2 fDCwME'})
    for cont in contacts:
        #для руководителя и телефона текст, для почты и сайта - ссылка
        cont_text = cont.find('p', attrs = {'class' : 'styles__TypographyP-sc-1txyxb-4 QLiFf'})
        cont_link = cont.find('a', attrs = {'class' : 'styles__BuilderCardContactsBlockLink-erzrc1-4 hvhQXH'})
        if (cont_text) != None: #добавляем текст
            link_info.append(cont_text.get_text())
            continue
        if (cont_link) != None: #добавляем ссылку и очищаем от автоматического добавления 'mailto:'
            if cont_link.get('href') != '':
                link_info.append(cont_link.get('href').replace('mailto:', ''))
            else:
                link_info.append('-')
            continue
        else:
            link_info.append('-')
    return link_info

***

## Извлечение данных

In [38]:
df_global = pd.DataFrame(columns=['Застройщик', 'ИНН', 'КПП', 'ОГРН', 'Полное наименование', "Юридический адрес", "Фактический адрес", "Руководитель", "Телефон", "Сайт", "E-mail"])
#извлекаем по 100 позиций для каждой из 46 страниц
for i in range(0,46):
    url = f'https://наш.дом.рф/сервисы/единый-реестр-застройщиков?page={i}&limit=4600' #лимит = 4600
    df = get_info_page(url) #парсинг данных для страницы
    #добавляем в единый датафрейм
    df_global = pd.concat([df_global, df], axis=0, sort=False)

['ООО 1 СХ', '3906245291', '390601001', '1113926031922', 'Общество с ограниченной ответственностью 1 СТРОИТЕЛЬНЫЙ ХОЛДИНГ', 'Калининградская область, город Калининград, улица Воскресенская дом 6 квартира 30', 'обл Калининградская, г Калининград, улица Генерала Озерова, д. 17Б', 'Пашковский Павел Петрович', '+7 (401) 230-59-00', 'http://www.jk-yutta.ru', 'ckmonolit2007@yandex.ru']
11
['ООО 10 КВАРТАЛ', '5050102110', '500701001', '1135050000447', 'Общество с ограниченной ответственностью 10 КВАРТАЛ', 'Московская область, город Дмитров, улица Весенняя дом 29 корпус - строение - комната 2', 'обл Московская Дмитровский город Дмитров село Озерецкое д. 29 комната 2 50', 'Меламуд Александр Романович', '+7 (495) 107-91-24', 'http://www.mechta.su', 'resnov@cmft.ru']
11
['ООО 168 КВАРТАЛ', '7703769272', '770301001', '1127746426136', 'Общество с ограниченной ответственностью 168 КВАРТАЛ', 'г Москва, набережная Пресненская дом 8 строение 1 помещение 21 комната 1', 'город Москва, набережная Пресненс

['АО ААК ВОЛОГДААГРОСТРОЙ', '3525000888', '352501001', '1023500886386', 'Акционерное общество АКЦИОНЕРНАЯ АГРОСТРОИТЕЛЬНАЯ КОМПАНИЯ ВОЛОГДААГРОСТРОЙ', 'Волгоградская область, г Вологда, улица Галкинская дом 24', 'обл Вологодская, г Вологда, улица Галкинская, д. 24', 'Корытин Владимир Николаевич', '+7 (817) 274-79-00', 'http://vologdaagrostroy.ru', 'vologdaagrostroy@yandex.ru']
11
['ООО АБСОЛЮТ', '7448189898', '744801001', '1167456064236', 'Общество с ограниченной ответственностью АБСОЛЮТ', 'Челябинская область, город Челябинск, проспект Победы дом 390 корпус А офис 16А', 'обл Челябинская город Челябинск проспект Победы д. 390 кор. А оф. 16А', 'Емельянов Юрий Васильевич', '+7 (351) 272-03-02', 'http://www.absolyt74.ru', 'stroikomplekt2@mail.ru']
11
['ООО АВА ДЕВЕЛОПМЕНТ ПЛЮС', '2312266578', '231201001', '1172375095661', 'Общество с ограниченной ответственностью АВА ДЕВЕЛОПМЕНТ ПЛЮС', 'Краснодарский край, город Краснодар, улица Бородинская дом 14 помещение 318', 'край Краснодарский город

['ЖСК АВТОНОМНАЯ', '1831188385', '184001001', '1171832029918', 'Жилищный или жилищно-строительный кооператив АВТОНОМНАЯ', 'Удмуртская Республика, город Ижевск, улица Союзная дом 99 квартира 27', 'Респ Удмуртская город Ижевск улица Союзная д. 99 кв. 27', '-', '+7 (963) 546-00-30', 'http://avtonom.ru', 'avtonom2017@bk.ru']
11
['ООО АВТОПОРТАЛ', '5610153825', '561001001', '1135658014854', 'Общество с ограниченной ответственностью АВТОПОРТАЛ', 'Оренбургская область, город Оренбург, улица Донецкая дом 2', '460001 обл Оренбургская г Оренбург ул Донецкая д. 2 56', '-', '-', 'http://www.наш.дом.рф', '1@mail.ru']
11
['ООО АВТОПРЕМИУМ', '4401154990', '440101001', '1144401008476', 'Общество с ограниченной ответственностью АВТОПРЕМИУМ', 'Костромская область, город Кострома, улица Самоковская дом 10А офис 2', 'обл Костромская город Кострома ул Самоковская д. 10А оф. 2', 'Марченкова Светлана Константиновна', '+7 (494) 244-04-40', 'http://www.kfk1.ru', 'info@kfk1.ru']
11
['ООО АВТОСПЕЦГАРАНТ', '43451

['ООО АКАДЕМГРАД', '1831176380', '183101001', '1151832015830', 'Общество с ограниченной ответственностью АКАДЕМГРАД', 'Удмуртская Республика, город Ижевск, улица Холмогорова дом 83 квартира 88', 'Респ Удмуртская город Ижевск улица Пушкинская д. 175 оф. 16', 'Мокрецов Михаил Александрович', '+7 (919) 919-20-77', 'http://академград18.рф', 'src10@mail.ru']
11
['ООО АКАДЕММЕДСТРОЙ', '5408305030', '540801001', '1135476161083', 'Общество с ограниченной ответственностью АКАДЕММЕДСТРОЙ', 'Новосибирская область, город Новосибирск, улица Разъездная дом 10', 'Новосибирская область, город Новосибирск, улица Разъездная, 10', 'Корсаков Александр Павлович', '+7 (913) 916-76-40', 'http://академмедстрой.рф', 'arhstroy1@mail.ru']
11
['АО АКАДЕМПАРК', '5408252116', '540801001', '1075473006905', 'Акционерное общество ТЕХНОПАРК НОВОСИБИРСКОГО АКАДЕМГОРОДКА', 'Новосибирская область, город Новосибирск, улица Николаева дом 12 помещение этаж 12', 'Новосибирская область, город Новосибирск, улица Николаева 12, э

['ООО АЛТАЙ-СТРОЙ', '4345090468', '434501001', '1044316571694', 'Общество с ограниченной ответственностью АЛТАЙ-СТРОЙ', 'Кировская область, город Киров, улица Некрасова дом 73', 'обл Кировская г Киров ул Некрасова д. 73', 'Сунцов Сергей Владимирович', '+7 (833) 271-22-25', 'http://altai-stroi.org', 'kvart@alt43.ru']
11
['ООО АЛТПРОМСНАБ', '2223032641', '540601001', '1022201142920', 'Общество с ограниченной ответственностью АЛТПРОМСНАБ', 'Новосибирская область, город Новосибирск, улица Семьи Шамшиных дом 30 помещение 88', 'обл Новосибирская г. Новосибирск ул. Семьи Шамшиных\xa0 д. 30 помещение 88', 'Орлов Александр Степанович', '+7 (903) 912-19-84', 'http://алтпромснаб.рф', 'altpromsnab@inbox.ru']
11
['ООО АЛЬБАТРОС', '7811385731', '781001001', '1077847610323', 'Общество с ограниченной ответственностью АЛЬБАТРОС', 'город Санкт-Петербург, площадь Конституции дом 7 корпус А помещение 18-H офис 708', 'г Санкт-Петербург площадь Конституции д. 7 кор. А помещение 18-H оф. 708', 'Абдиева Жумаг

...

['АО ЭКЗ', '5047001778', '504701001', '1025006175700', 'Акционерное общество ЭКСПЕРИМЕНТАЛЬНЫЙ КЕРАМИЧЕСКИЙ ЗАВОД', 'Московская область, Город Химки, мкр. Подрезково, улица Центральная дом 2/5 офис каб. 62', 'Московская область, город Химки, микрорайон Подрезково, улица Центральная.2, каб.62', 'Турта Сергей Александрович', '+7 (495) 574-60-74', 'http://6660444.ru', 'ekzokc@mail.ru']
11
['ООО ЭКО ПЛЮС', '2539059167', '253901001', '1032502134719', 'Общество с ограниченной ответственностью ЭКО ПЛЮС', 'Приморский край, город Владивосток, улица Кирова дом 23 помещение 306Б.', 'край Приморский г Владивосток ул Кирова д. 23 помещение 306Б', 'Кулешова Вероника Валентиновна', '+7 (423) 220-50-13', 'http://жкэкосити.рф', 'ekoplus_office@bk.ru']
11
['ООО ЭКО ПРОМ', '2465274914', '246501001', '1122468037427', 'Общество с ограниченной ответственностью ЭКО ПРОМ', 'Красноярский край, город Красноярск, улица Ястынская дом 29а', 'край Красноярский город Красноярск улица 78 Добровольческой бригады д. 40

['ООО ЭЛИТСТРОЙ', '6829136090', '682901001', '1176820011576', 'Общество с ограниченной ответственностью ЭЛИТСТРОЙ', 'Тамбовская область, Город Тамбов, улица С.Ценского дом 36', 'Тамбовская область, Город Тамбов, улица С.Ценского дом 36', '-', '+7 (475) 279-67-48', 'http://www.наш.дом.рф', 'taranina56@mail.ru']
11
['ООО ЭЛИТСТРОЙ', '5040134024', '504001001', '1145040013062', 'Общество с ограниченной ответственностью ЭЛИТСТРОЙ', 'Московская область, город Жуковский, улица Праволинейная дом 33 офис 307', 'обл Московская г Жуковский улица Праволинейная д. 33 оф. 307', 'Фролов Александр Викторович', '+7 (495) 197-83-77', 'http://www.jk-ng.ru', 'elitstroy00@list.ru']
11
['ООО ЭЛИТЮГСТРОЙ', '2630049320', '263001001', '1172651026008', 'Общество с ограниченной ответственностью ЭЛИТЮГСТРОЙ', 'Ставропольский край, посёлок Первомайский, улица Московская дом 16', 'край Ставропольский Минераловодский поселок Первомайский д. 16', 'Попов Янис Николаевич', '+7 (905) 410-12-57', 'http://Элитюгстрой.рф',

['ООО ЮБК СПЕЦСТРОЙ', '9103003835', '920101001', '1149102036511', 'Общество с ограниченной ответственностью ЮБК СПЕЦСТРОЙ', 'город Севастополь, улица Молодых строителей дом 43', 'г Севастополь город Севастополь улица Молодых строителей д. 43', 'Гончаров Александр Евгеньевич', '+7 (869) 292-92-92', 'http://www.ga-rant.com', 'ceo@ga-rant.com']
11
['ООО ЮГ ИНВЕСТ СТРОЙ', '2310112935', '010701001', '1142310004760', 'Общество с ограниченной ответственностью ЮГ ИНВЕСТ СТРОЙ', 'Адыгея Республика, посёлок городского типа Энем, переулок Ильницкого дом 3 литера Г6', 'Респ Адыгея Тахтамукайский поселок городского типа Энем пер. Ильницкого д. 3 стр. Литер Г6', 'Мурадян Андраник Арменович', '+7 (988) 246-90-91', 'http://www.gk-kuban.ru', 'uo@gk-kuban.ru']
11
['ООО ЮГ-ГАРАНТСТРОЙ', '2308194144', '231001001', '1122308011231', 'Общество с ограниченной ответственностью ЮГ-ГАРАНТСТРОЙ', 'Краснодарский край, город Краснодар, улица Северная/Коммунаров дом 372/216 офис 6', 'край Краснодарский г Краснодар С

['ООО ЮЖС', '6162004539', '616101001', '1026103056814', 'Общество с ограниченной ответственностью ЮЖС', 'Ростовская область, город Ростов-на-Дону, улица Бодрая дом 41', 'обл Ростовская город Ростов-на-Дону улица Бодрая д. 41', 'Гиниятулина Наталья Салаватовна', '+7 (863) 231-22-88', 'http://ugs-stroi.ru', 'ugstroy@donpac.ru']
11
['АО ЮИТ МР', '5005053622', '500501001', '1115005000990', 'Акционерное общество ЮИТ МОСКОВСКИЙ РЕГИОН', 'Московская область, город Воскресенск, улица Московская дом 41', 'обл Московская Люберецкий г Люберцы ул Красная д. 4', 'Старостина Ольга Михайловна', '+7 (495) 229-68-27', 'http://www.yit-mo.ru', 'secr@yit.ru']
11
['АО ЮИТ САНКТ-ПЕТЕРБУРГ', '7814313164', '781401001', '1057810048350', 'Акционерное общество ЮИТ САНКТ-ПЕТЕРБУРГ', 'город Санкт-Петербург, проспект Приморский дом 54 корпус 1 литера А', 'Город Санкт-Петербург, проспект Приморский 54к1, литера А', 'Хелпполайнен Теему Тапани Тапани', '+7 (812) 336-37-57', 'http://www.yitspb.ru', 'yitdom.spb@yit.ru']

['АО ЯРКИЙ МИР', '7722442868', '502901001', '1187746257038', 'Акционерное общество ЯРКИЙ МИР', 'Московская область, город Мытищи, улица Борисовка дом 4 помещение X комната 22', 'Московская область, город Мытищи, улица Борисовка, д.4, комн.22, пом. X', 'Зиновьев Александр Львович', '+7 (495) 500-00-20', 'http://www.bw-estate.ru', 'info@bw-estate.ru']
11
['ООО ЯРКИЙ-12', '0278908839', '027801001', '1150280055090', 'Общество с ограниченной ответственностью СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ЯРКИЙ-12', 'Башкортостан Республика, город Уфа, бульвар Хадии Давлетшиной дом 11 корпус А офис 206', 'Респ Башкортостан город Уфа бульвар Хадии Давлетшиной д. 11 кор. А оф. 206', 'Самойленко Елена Александровна', '+7 (347) 285-85-83', 'http://www.yarkii12.ru', 'mail@trest3rb.ru']
11
['ООО ЯРКИЙ-2А', '0278915378', '027801001', '1160280069388', 'Общество с ограниченной ответственностью СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ЯРКИЙ-2А', 'Башкортостан Республика, город Уфа, бульвар Хадии Давлетшиной дом 11 корпус А офис 

****

Результирующая база данных:

In [39]:
df_global

,Застройщик,ИНН,КПП,ОГРН,Полное наименование,Юридический адрес,Фактический адрес,Руководитель,Телефон,Сайт,E-mail
0,ООО 1 СХ,3906245291,390601001,1113926031922,Общество с ограниченной ответственностью 1 СТР...,"Калининградская область, город Калининград, ул...","обл Калининградская, г Калининград, улица Гене...",Пашковский Павел Петрович,+7 (401) 230-59-00,http://www.jk-yutta.ru,ckmonolit2007@yandex.ru
1,ООО 10 КВАРТАЛ,5050102110,500701001,1135050000447,Общество с ограниченной ответственностью 10 КВ...,"Московская область, город Дмитров, улица Весен...",обл Московская Дмитровский город Дмитров село ...,Меламуд Александр Романович,+7 (495) 107-91-24,http://www.mechta.su,resnov@cmft.ru
2,ООО 168 КВАРТАЛ,7703769272,770301001,1127746426136,Общество с ограниченной ответственностью 168 К...,"г Москва, набережная Пресненская дом 8 строени...","город Москва, набережная Пресненская, д. 8, ст...",Шпакова Ирина Николаевна,+7 (495) 988-98-21,http://www.kvartal-pobedy.ru,info@m1develop.ru
3,ЖСК 212-2,6154572922,615401001,1116154005923,Жилищный или жилищно-строительный кооператив 2...,"Ростовская область, город Таганрог, улица Своб...",обл Ростовская город Таганрог улица Сергея Шил...,Снимщикова Ольга Викторовна,+7 (961) 285-90-10,http://taganrogjsk.wixsite.com,kirpol76@gmail.com
4,АО 2МЕН ГРУПП,7701651356,720301001,1067746424899,Акционерное общество 2МЕН ГРУПП,"Тюменская область, город Тюмень, улица Комсомо...","Тюменская область, город Тюмень, улица Комсомо...",Киселев Сергей Михайлович,+7 (345) 279-18-88,http://2mengroup.ru,info@2mengroup.ru
...,...,...,...,...,...,...,...,...,...,...,...
4550,ООО СЗ ИНВЕСТТРАСТ 1,7751137450,775101001,1187746010979,Общество с ограниченной ответственностью ‎СПЕЦ...,"город Москва, ул Футбольная дом 17 этаж 3 офис...",", г.Москва, ул. Футбольная, д. 17 этаж 3, офис 1",Гаврилов Илья Григорьевич,+7 (499) 962-49-62,http://investtrust1.com,info@investtrust.com
4551,ООО СЗ ПУШКИНСКИЙ,7813620821,781301001,1187847271281,Общество с ограниченной ответственностью ‎СПЕЦ...,"город Санкт-Петербург, проспект Большой П.С. д...","г Санкт-Петербург, проспект Большой П.С., д. 4...",Возиянов Михаил Владимирович,+7 (812) 449-50-39,http://pushkinskii.ru,info@pushkinskii.ru
4552,ООО СЗ СОЗИДАНИЕ,7810718995,781401001,1187847000164,Общество с ограниченной ответственностью СПЕЦИ...,", набережная Ушаковская дом 3 корпус 1 строени...",", Город Санкт-Петербург, набережная Ушаковская...",Еременко Илья Анатольевич,+7 (812) 335-51-11,http://www.setlcity.ru,Info@setlcity.ru
4553,ООО СЗ СТРОЙ-АМОНД,6314046254,631401001,1196313012180,Общество с ограниченной ответственностью ‎СПЕЦ...,"Самарская область, город Самара, улица Казачья...",обл Самарская г Самара ул Казачья д. 2а,Хугаев Ростик Ирбегович,+7 (846) 311-00-25,http://www.stroy-amond.ru,stroy-amond@mail.ru


***

Наконец, извлекаем файл в эксель-формате.

In [41]:
df_global.to_excel('C:/path_to_file/file_name.xlsx')